In [1]:
import numpy as np
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10

train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')

X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

#from matpotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout ,Input, Flatten ,Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras import backend as K

Using TensorFlow backend.


# Tenserflow testing


In [2]:
import tensorflow as tf

In [ ]:
Hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(Hello).decode())

Using TensorFlow backend.


# Define Our Model


In [3]:
def myFirstModel():
    mymodel = Sequential()
    # conv >maxpool>dropout layer 1
    
    mymodel.add(Conv2D(64,kernel_size=(3,3),activation ='relu', input_shape=(75,75,3)))
    mymodel.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    
    mymodel.add(Dropout(0.2))
     #Conv Layer 3
    mymodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    mymodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    mymodel.add(Dropout(0.2))
    
    # conv 
    
    mymodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    mymodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   
    mymodel.add(Dropout(0.2))
    
    
    #Conv Layer 4
    mymodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    mymodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    mymodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    mymodel.add(Flatten())

    #Dense Layers
    mymodel.add(Dense(512))
    mymodel.add(Activation('relu'))
    mymodel.add(Dropout(0.2))

    #Dense Layer 2
    mymodel.add(Dense(256))
    mymodel.add(Activation('relu'))
    mymodel.add(Dropout(0.2))

    #Sigmoid Layer
    
    mymodel.add(Dense(1))
    mymodel.add(Activation('sigmoid'))
    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    mymodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    mymodel.summary()
    return mymodel

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)
    
    

In [ ]:
#train[train.inc_angle=='na']['inc_angle']=0

datagen = ImageDataGenerator(featurewise_center=True,
    samplewise_center=True,
    featurewise_std_normalization=True,
    samplewise_std_normalization=True,
    zca_whitening=True,
    zca_epsilon=1e-6,
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.,
    zoom_range=0.1,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format=K.image_data_format())
datagen.fit(X_train)


target_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

import os
mymodel=myFirstModel()
mymodel.fit_generator(datagen.flow(X_train_cv, y_train_cv, batch_size=32),
                    steps_per_epoch=2**14 / 32, epochs=30,verbose=1,
        validation_data=(X_valid, y_valid),callbacks=callbacks)



C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [47]:
import os
nmodel=myFirstModel()
nmodel.fit_generator(datagen.flow(X_train_cv, y_train_cv, batch_size=32),
                    steps_per_epoch=2**14 / 32, epochs=30,verbose=1,
        validation_data=(X_valid, y_valid),callbacks=callbacks)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 36, 36, 64)        256       
_________________________________________________________________
dropout_35 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 17, 17, 128)       512       
__________

KeyboardInterrupt: 

In [16]:
newmodel.load_weights(filepath=file_path)
score =newmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

401/401 [==============================] - 6s 15ms/step
Test loss: 0.271501764469
Test accuracy: 0.877805486879


In [11]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=newymodel.predict_proba(X_test)

In [13]:
predicted_test

array([[ 0.11866949],
       [ 0.46353978],
       [ 1.        ],
       ..., 
       [ 0.0244566 ],
       [ 0.99312347],
       [ 0.00383051]], dtype=float32)

In [17]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('simplemodelagain.csv', index=False)

In [34]:
train

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0
5,"[-20.769371, -20.769434, -25.906025, -25.90602...","[-29.288746, -29.712593, -28.884804, -28.88480...",b51d18b5,36.9034,1
6,"[-26.673811, -23.666162, -27.622442, -28.31768...","[-24.557735, -26.97868, -27.622442, -29.073456...",31da1a04,34.4751,1
7,"[-24.989119, -27.755224, -25.817074, -24.98927...","[-27.755173, -26.732174, -28.124943, -31.83772...",56929c16,41.1769,0
8,"[-17.146641, -17.146572, -17.994583, -19.44553...","[-25.733608, -24.472507, -24.710424, -22.77215...",525ab75c,35.7829,0
9,"[-24.020853, -23.551275, -27.18819, -29.126434...","[-28.702518, -33.563324, -29.571918, -29.12643...",192f56eb,43.3007,0


# ResNet50

In [46]:
2**9

512